In [ ]:
from sparsecnn_with_prediction_model import *
from losses import *


In [ ]:
from keras import backend as K
import tensorflow as tf
from keras.layers import Layer, Lambda, Convolution2D, Concatenate, Add, Multiply, MaxPooling2D, ReLU, Activation
from keras.models import Model, Input


##### SPARSECONV MODEL #####


###  A simple Bias-only layer implementation
###   we assume a channel-last layout
class Bias(Layer):

    def __init__(self, **kwargs):
        super(Bias, self).__init__(**kwargs)

    def build(self, input_shape):

        self.bias = self.add_weight(name='bias',
                            shape=(input_shape[1],input_shape[2],input_shape[3]),
                            initializer='zeros',
                            trainable=True)

        super(Bias, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return x + self.bias

    def compute_output_shape(self, input_shape):
        return input_shape
        
    
###  Returns a sparse convolution block as
###  described in 
###  Jonas Uhrig et al. "Sparsity Invariant CNNs", Figure 2 (right)
def sparse_conv_block( I, M, ksize, depth ):

    IM = Multiply( )([I,M] )

    IMc = Convolution2D( depth, ksize, padding='same', use_bias=False, activation=None) ( IM )

    # A hack to convolve with a fixed "ones" kernel
    Mc = Convolution2D( 1, ksize, padding='same', use_bias=False, activation=None, 
                        kernel_initializer = 'ones',
                        trainable=False) ( M )


    Inorm = Lambda( lambda x: x[0]/(x[1]+1E-5) )( [IMc,Mc] )
    InormB = Bias( )(Inorm)

    #Mpooled = MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same')(M)
    Mpooled = MaxPooling2D(pool_size=ksize, strides=(1,1), padding='same')(M)

    return InormB, Mpooled

    
def create_sparseconv_256_model( input_layer=None ):
    
    if input_layer is None:
        input_data = Input( shape=(256,256,2) )
    else:
        input_data = input_layer
        
    I0 = Lambda( lambda x: K.expand_dims(x[:,:,:,0],axis=-1), name="I0" )( input_data )
    M0 = Lambda( lambda x: K.expand_dims(x[:,:,:,1],axis=-1), name="M0" )( input_data )
    
    I,M = sparse_conv_block( I0, M0, (11,11), 16)
    I = Activation('sigmoid')( I )
    I,M = sparse_conv_block( I, M, (7,7), 16)
    I = Activation('sigmoid')( I )
    I,M = sparse_conv_block( I, M, (5,5), 16)
    I = Activation('sigmoid')( I )
    I,M = sparse_conv_block( I, M, (3,3), 16)
    I = Activation('sigmoid')( I )
    I,M = sparse_conv_block( I, M, (3,3), 16)
    I = Activation('sigmoid')( I )

    I,M = sparse_conv_block( I, M, (1,1), 1)
    O = Activation('linear', name="sc_out")( I )
    
    model = Model(inputs=input_data, outputs=O )    
    return model



##### MODEL WITH PREDICTION (prev, curr, next)#####

def compute_data_mask( data ):
    
    sparse_data = tf.expand_dims(data, axis=-1)
    mask = tf.cast( tf.math.logical_not( tf.math.is_nan(sparse_data)), tf.float32)
    
    sparse_data = tf.math.multiply_no_nan(sparse_data, mask) # NaN * 0 = 0
    
    return tf.concat((sparse_data, mask), axis=3)


def spec_predict( l, phdiff ):
    
    l_spec = tf.signal.fft2d(tf.complex(l[:,:,:,0], 0.0))
    l_spec =  tf.multiply(l_spec, phdiff)
    
    l_out = tf.math.real(tf.signal.ifft2d(l_spec)) 
    l_out = tf.expand_dims(l_out, axis=-1)
    
    return l_out
    

def create_model_with_prediction( sparsecnn_trainable=False ):
    
    model_sparsecnn = create_sparseconv_256_model()

    if not sparsecnn_trainable:
        for l in model_sparsecnn.layers:
            l.trainable = False
    
    input_data = Input( shape=(256,256,3) )
    input_ph_diff_matrix = Input( shape=(256,256,4) ) # prev_real, prev_imag, next_real, next_imag
    
    IpMp = Lambda( lambda x: compute_data_mask(x[:,:,:,0]), name="IpMp" )( input_data )
    IcMc = Lambda( lambda x: compute_data_mask(x[:,:,:,1]), name="IcMc" )( input_data )
    InMn = Lambda( lambda x: compute_data_mask(x[:,:,:,2]), name="InMn" )( input_data )
    
    Op = model_sparsecnn( IpMp )
    Oc = model_sparsecnn( IcMc )
    On = model_sparsecnn( InMn )
    
    ph_diff_prev = Lambda( lambda x: tf.complex(x[:,:,:,0], x[:,:,:,1])) (input_ph_diff_matrix)
    ph_diff_next = Lambda( lambda x: tf.complex(x[:,:,:,2], x[:,:,:,3])) (input_ph_diff_matrix)
    
    Oc = Lambda( lambda x: x, name="curr" )( Oc )
    
    Op_pred = Lambda( lambda x: spec_predict( x[0], x[1]), name="predict_prev" )( [Op, ph_diff_prev] )
    On_pred = Lambda( lambda x: spec_predict( x[0], x[1]), name="predict_next" )( [On, ph_diff_next] )
    
    Oc = Concatenate()([Op_pred, Oc, On_pred])
        
    Oc = Convolution2D( 16, (5,5), padding='same', use_bias=True, activation="sigmoid") ( Oc )
    Oc = Convolution2D( 16, (3,3), padding='same', use_bias=True, activation="sigmoid") ( Oc )
    Oc = Convolution2D( 8, (3,3), padding='same', use_bias=True, activation="sigmoid") ( Oc )
    Oc = Convolution2D( 1, (1,1), padding='same', use_bias=True, activation="linear") ( Oc )
        
    model = Model(inputs = [input_data,input_ph_diff_matrix], outputs=Oc )
    return model
    
    

# create model
model = create_model_with_prediction()
print(model.summary())

# load weights
model.load_weights('modeldata/prediction_v1_2020-09-22_16-07-17.h5')


In [ ]:
# prediction test
import h5py
import numpy as np
from SparseSeriesDataGenerator import SparseSeriesDataGenerator

h5_data = h5py.File('data/waves_256x256_7fps_pred.h5', 'r')
sceneries  = [k for k in h5_data.keys()]
d0 = h5_data[sceneries[0]]

dt = np.array(d0['dt']).item(0)
KxKy = np.array( d0['KxKy'] )
angle = np.array(d0['waveangle']).item(0)
test_data = d0["train"]

test_sampling = (0.03, 0.05)


test_generator = SparseSeriesDataGenerator(test_data, test_sampling, KxKy, angle, dt, batch_size=32)

x, y_gt = test_generator[0]
y_pred = model.predict(x)

print(x[0].shape)
print(x[1].shape)
print(y_gt.shape)


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

for idx in range(0, 5 ):
    
    sparse_data = x[0][idx,:,:,1];
    
    fig =  plt.figure(figsize=(30, 7))
    
    fig.add_subplot(1, 3, 1);
    plt.pcolor(sparse_data)
    #plt.clim(zminmax[0],zminmax[1])
    plt.colorbar()
    plt.title('Sparse data curr (input)')

    '''fig.add_subplot(1, 6, 2)
    plt.pcolor(masked_data[idx,:,:,1])
    #plt.clim(zminmax[0],zminmax[1])
    plt.colorbar()
    plt.title('binary mask')'''
    
    fig.add_subplot(1, 3, 2);
    plt.pcolor(y_pred[idx,:,:,0])
    #plt.clim(zminmax[0],zminmax[1])
    plt.colorbar()
    plt.title('network output')
    
    fig.add_subplot(1, 3, 3)
    plt.pcolor(y_gt[idx,:,:,0])
    #plt.clim(zminmax[0],zminmax[1])
    plt.colorbar()
    plt.title('GT curr')